#### Notebook description
- this notebooks creates a raw scenario (where no project cards are applied) from standard network files

#### Basic package & directory settings
- network_wrangler branch: [bicounty_ml_connectors](https://github.com/wsp-sag/network_wrangler/tree/bicounty_ml_connectors)
- Lasso branch: [bicounty_emme](https://github.com/wsp-sag/Lasso/tree/bicounty_emme)

In [ ]:
import os
import sys
import yaml
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger
from network_wrangler.transitnetwork import DotDict

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util
from lasso import bicounty

import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.INFO)

%load_ext autoreload
%autoreload 2

In [ ]:
data_dir = "../data/"
lasso_dir = "../software/Lasso/"
parameters = Parameters(lasso_base_dir = lasso_dir)

standard_net_dir = f"{data_dir}/scenario_files/standard_networks/version_00"
interim_data_dir = f"{data_dir}/scenario_files/interim/"

#### Load version00 standard network

In [ ]:
link_file = os.path.join(standard_net_dir, 'step8_links.json')
node_file = os.path.join(standard_net_dir, 'step8_nodes.geojson')
shape_file = os.path.join(standard_net_dir, 'step8_shapes.geojson')

roadway_net = RoadwayNetwork.read(
    link_filename = link_file, 
    node_filename = node_file, 
    shape_filename = shape_file
)
# time: 5.5 min

In [ ]:
transit_net = TransitNetwork.read(feed_path = os.path.join(standard_net_dir, 'transit'))

#### calculate network attributes

In [ ]:
r_net = mtc.determine_number_of_lanes(
    roadway_network = roadway_net,
    parameters=parameters,
    sj_osm_lanes_attributes = os.path.join(interim_data_dir, 'interim_version_01', 'osm_lanes.csv'),
    sjmodel_lanes_file = os.path.join(interim_data_dir, 'interim_version_01', 'sjmodel_legacy_lanes.csv'),
    ccta_lanes_file = os.path.join(interim_data_dir, 'interim_version_02', 'cctamodel_legacy_lanes.csv'),
    actc_lanes_file = os.path.join(interim_data_dir, 'interim_version_02', 'actcmodel_legacy_lanes.csv'),
    network_variable = 'lanes'
)
r_net.links_df.lanes.value_counts()
# time: 5 min

In [ ]:
r_net = mtc.calculate_facility_type(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'ft'
)
r_net.links_df.ft.value_counts()
# time: 2 min

In [ ]:
r_net = mtc.calculate_useclass(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'useclass'
)
r_net.links_df.useclass.value_counts()

In [ ]:
r_net = mtc.calculate_assignable(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'assignable'
)
r_net.links_df['assignable'] = np.where(
    (r_net.links_df.drive_access == 1) & (r_net.links_df.roadway != 'service') & (r_net.links_df.county == 'San Joaquin'),
    1,
    r_net.links_df['assignable']
)
r_net.links_df['assignable'].value_counts()

In [ ]:
r_net = mtc.calculate_transit(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'transit'
)
r_net.links_df.transit.value_counts()

In [ ]:
r_net = mtc.calculate_farezone(
    roadway_network = r_net,
    transit_network = transit_net,
    parameters = parameters,
    network_variable = 'farezone',
)
r_net.nodes_df.farezone.value_counts()

In [ ]:
r_net = mtc.calculate_bike(
    roadway_network = r_net,
    parameters = parameters,
    actc_bike_file = os.path.join(interim_data_dir, 'interim_version_02', 'actcmodel_legacy_bike.csv'),
)
r_net.links_df.nmt2010.value_counts(dropna=False)
# time: 1 min

#### create scenario 00

In [ ]:
curr_scenario = Scenario.create_scenario(
    base_scenario = {"road_net": r_net, "transit_net": transit_net}
)

curr_scenario.road_net.links_df["id"] = curr_scenario.road_net.links_df["shstGeometryId"]
curr_scenario.road_net.links_df.drop('length',axis = 1, inplace = True)

version_00_scenario_filename = os.path.join(data_dir, 'scenario_files', 'scenario_pickle', 'raw_scenario.pickle')
pickle.dump(curr_scenario, open(version_00_scenario_filename, 'wb'))
# time: 1 min

In [ ]:
print(len(curr_scenario.road_net.links_df))
print(len(curr_scenario.road_net.nodes_df))

In [ ]:
pd.crosstab(curr_scenario.road_net.links_df.ft, curr_scenario.road_net.links_df.roadway)